# DNBLab Jupyter Notebook Tutorial

## SRU - Schnittstellenabfrage, Datenauslieferung und Ergebnisanzeige

Dieses DNBLab-Tutorial beschreibt eine Beispielabfrage über die SRU-Schnittstelle mit Python. In der Jupyter Notebook Umgebung kann der dokumentierte Code direkt ausgeführt und angepasst werden. Das Tutorial umfasst die exemplarische Anfrage und Ausgabe der Daten in MARC21-xml zur weiteren Verarbeitung.

Die Daten können über die SRU-Schnittstelle als XML-Antwort in folgenden Formaten ausgeliefert werden: 
* MARC21-xml http://www.loc.gov/standards/marcxml/schema/MARC21slim.xsd
* DNB Casual (oai_dc)	http://www.openarchives.org/OAI/2.0/oai_dc.xsd
* RDF (RDFxml) http://www.w3.org/2000/07/rdf.xsd

## Einrichten der Arbeitsumgebung <a class="anchor" id="Teil1"></a>

Um die Arbeitsumgebung für die folgenden Schritte passend einzurichten, sollten zunächst die benötigten Python-Bibliotheken importiert werden. Für Anfragen über die SRU-Schnittstelle wird BeautifulSoup https://www.crummy.com/software/BeautifulSoup/ und zur Verarbeitung der XML-Daten etree https://docs.python.org/3/library/xml.etree.elementtree.html verwendet. Mit Pandas https://pandas.pydata.org/ können Elemente aus dem MARC21-Format ausgelesen werden.

In [1]:
import requests
from bs4 import BeautifulSoup as soup
import unicodedata
from lxml import etree
import pandas as pd

## SRU-Abfrage mit Ausgabe in MARC21-xml<a class="anchor" id="Teil2"></a>

Die Funktion dnb_sru nimmt den Paramter "query" der SRU-Abfrage entgegen und liefert alle Ergebnisse als eine Liste von records aus.

In [2]:
def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content)
    records = xml.find_all('record', {'type':'Bibliographic'})
    
    if len(records) < 100:
        
        return records
    
    else:
        
        num_results = 100
        i = 101
        while num_results == 100:
            
            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content)
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            
        return records

## Durchsuchen eines MARC-Feldes<a class="anchor" id="Teil3"></a>

Die Funktion parse_records nimmt als Parameter jeweils ein Record entgegen und sucht über xpath die gewünschte Informationen (in diesem Fall IDN und Titel) heraus und liefert diese als dictionary zurück. 

In [3]:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #idn
    idn = xml.xpath("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
    
    # titel
    titel = xml.xpath("marc:datafield[@tag = '245']/marc:subfield[@code = 'a']", namespaces=ns)
    
    try:
        titel = titel[0].text
        #umfang = unicodedata.normalize("NFC", titel)
    except:
        titel = "unkown"
        
         # urn
    urn = xml.xpath("marc:datafield[@ind1 = ' ']/marc:subfield[@code = 'u']", namespaces=ns)
    
    try:
        urn = urn[0].text
        #urn = unicodedata.normalize("NFC", urn)
    except:
        urn = 'fail'
        
         # umfang
    umfang = xml.xpath("marc:datafield[@tag = '300']/marc:subfield[@code = 'a']", namespaces=ns)
    try:
        umfang = umfang[0].text
        #umfang = unicodedata.normalize("NFC", umfang)
    except:
        umfang = 'fail'
        
    meta_dict = {"idn":idn,
                 "titel":titel,
                 "urn":urn,
                 "umfang":umfang
                               }
    
    return meta_dict

Die verschiedenen Indexbezeichnungen stehen in https://services.dnb.de/sru/dnb?operation=explain&version=1.1 und können mittels CQL gezielt nach z.B. dem Titelstichwort "Klimawandel" in Kombination mit Standort "online frei verfügbar" abgefragt werden. Auf diese Art kann durch Anpassen des Codes auch nach anderen Begriffen oder Namen in beliebigen MARC-Feldern gesucht werden. 

In [4]:
records = dnb_sru('tit=Fischermädchen  and cod=t003 and location=onlinefree')
print(len(records), 'Ergebnisse')

1 Ergebnisse


In [5]:
print(records)

[<record type="Bibliographic" xmlns="http://www.loc.gov/MARC21/slim">
<leader>00000nam a2200000 c 4500</leader>
<controlfield tag="001">1076785042</controlfield>
<controlfield tag="003">DE-101</controlfield>
<controlfield tag="005">20171202132006.0</controlfield>
<controlfield tag="007">cr||||||||||||</controlfield>
<controlfield tag="008">150922s1896    gw |||||o|||| 00||||ger  </controlfield>
<datafield ind1=" " ind2=" " tag="015">
<subfield code="a">16,O12</subfield>
<subfield code="2">dnb</subfield>
</datafield>
<datafield ind1="7" ind2=" " tag="016">
<subfield code="2">DE-101</subfield>
<subfield code="a">1076785042</subfield>
</datafield>
<datafield ind1="7" ind2=" " tag="024">
<subfield code="2">urn</subfield>
<subfield code="a">urn:nbn:de:101:1-2015111111689</subfield>
</datafield>
<datafield ind1="3" ind2="2" tag="028">
<subfield code="b">Eterna (LC 0022)</subfield>
<subfield code="a">12013</subfield>
</datafield>
<datafield ind1="1" ind2="2" tag="028">
<subfield code="a">ET 6

## Beispielanzeige zur weiteren Bearbeitung <a class="anchor" id="Teil4"></a>

Mit der "Python Data Analysis Library" Pandas für Python werden die Ergebnisse als Dataframe ausgegeben. Die ersten 5 und letzten 5 Zeilen des Dataframes können mit dem Befehl "df = pd.DataFrame() angezeigt werden. Dabei können in ein Set Objekte beliebigen Datentyps gespeichert werden. 

In [5]:
output = [parse_record(record) for record in records]
df = pd.DataFrame(output)
df

,idn,titel,urn,umfang
0,1076785042,Das Fischermädchen - Leise zieht durch mein Gemüt,https://d-nb.info/1076785042/34,Digitalisat von 1 Schellackplatte


In [6]:
df.to_html(escape=False) 

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>idn</th>\n      <th>titel</th>\n      <th>urn</th>\n      <th>umfang</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>1076785042</td>\n      <td>Das Fischermädchen - Leise zieht durch mein Gemüt</td>\n      <td>https://d-nb.info/1076785042/34</td>\n      <td>Digitalisat von 1 Schellackplatte</td>\n    </tr>\n  </tbody>\n</table>'

In [7]:
import wget

!wget https://d-nb.info/1076785042/34

--2021-07-09 07:37:07--  https://d-nb.info/1076785042/34
Resolving d-nb.info (d-nb.info)... 193.175.100.223
Connecting to d-nb.info (d-nb.info)|193.175.100.223|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘34.1’

34.1                    [            <=>     ]  79.61M  16.4MB/s    in 4.4s    

2021-07-09 07:37:12 (18.2 MB/s) - ‘34.1’ saved [83476708]



In [ ]:
!unzip 34.1

Archive:  34.1
  inflating: 1111501032304_00020_00010_01_o.wav.xml  
  inflating: 1111501032304_00020_00010_01_o.wav  
  inflating: 1111501032304_00020_00010_01_o.tif  
replace 1111501032304_00020_00020_01_o.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Instantiating our first algorithm, loading some audio

Essentia has a selection of audio loaders:

AudioLoader: the most generic one, returns the audio samples, sampling rate and number of channels, and some other related information
MonoLoader: returns audio, down-mixed and resampled to a given sampling rate
EasyLoader: a MonoLoader which can optionally trim start/end slices and rescale according to a ReplayGain value
EqloudLoader: an EasyLoader that applies an equal-loudness filtering to the audio

In [ ]:
# first, we need to import our essentia module. It is aptly named 'essentia'!
import essentia

# as there are 2 operating modes in essentia which have the same algorithms,
# these latter are dispatched into 2 submodules:
import essentia.standard
import essentia.streaming

# let's have a look at what is in there
print(dir(essentia.standard))

# you can also do it by using autocompletion in IPython, typing "essentia.standard." and pressing Tab

In [ ]:
loader = essentia.standard.MonoLoader(filename='1111501032304_00020_00020_01_o.wav')

# and then we actually perform the loading:
audio = loader()

In [ ]:
import IPython
IPython.display.Audio('1111501032304_00020_00020_01_o.wav')

By default, the MonoLoader will output audio with 44100Hz sample rate downmixed to mono. To make sure that this actually worked, let's plot a 1-second slice of audio, from t = 1 sec to t = 2sec:

In [ ]:
# pylab contains the plot() function, as well as figure, etc... (same names as Matlab)
from pylab import plot, show, figure, imshow
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15, 6) # set plot sizes to something larger than default

plot(audio[1*44100:2*44100])
plt.title("This is how the 2nd second of this audio looks like:")
show() # unnecessary if you started "ipython --pylab"

Computing spectrum, mel bands energies, and MFCCs
So let's say that we want to compute spectral energy in mel bands and the associated MFCCs for the frames in our audio.

We will need the following algorithms: Windowing, Spectrum, MFCC. For windowing, we'll specify to use Hann window.

In [ ]:
from essentia.standard import *
w = Windowing(type = 'hann')
spectrum = Spectrum()  # FFT() would return the complex FFT, here we just want the magnitude spectrum
mfcc = MFCC()

In [ ]:
frame = audio[6*44100 : 6*44100 + 1024]
spec = spectrum(w(frame))
mfcc_bands, mfcc_coeffs = mfcc(spec)

plot(spec)
plt.title("The spectrum of a frame:")
show()

plot(mfcc_bands)
plt.title("Mel band spectral energies of a frame:")
show()

plot(mfcc_coeffs)
plt.title("First 13 MFCCs of a frame:")
show()